# Feature Engineering

## 1. Import libraries

In [31]:
import numpy as np
# !pip install streamlit
import streamlit as st
import pandas as pd

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
	OneHotEncoder,
	OrdinalEncoder,
	StandardScaler,
	MinMaxScaler,
	PowerTransformer,
	FunctionTransformer
)
from sklearn.pipeline import Pipeline
from feature_engine.encoding import RareLabelEncoder

from feature_engine.outliers import Winsorizer
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.encoding import (
	RareLabelEncoder,
	MeanEncoder,
	CountFrequencyEncoder
)

import matplotlib.pyplot as plt

import warnings

## 2. Display Setting

In [20]:
pd.set_option("display.max_columns", None)

In [21]:
#sklearn will return dataframe 
sklearn.set_config(transform_output= "pandas")

In [22]:
warnings.filterwarnings("ignore")

## 3. Read the data 

In [59]:
file_path = file_path = r"/Users/sudhirjoon/Library/Mobile Documents/com~apple~CloudDocs/Uni_Mannheim/Nitish/Project_4_CLASSIFICATION WITH AN ACADEMIC SUCCESS DATASET/Data/train_clean.csv"
train_clean = pd.read_csv(file_path)
train_clean.head()

,marital_status,application_mode,application_order,course,daytime/evening_attendance,previous_qualification,previous_qualification_(grade),nacionality,mother's_qualification,father's_qualification,mother's_occupation,father's_occupation,admission_grade,displaced,educational_special_needs,debtor,tuition_fees_up_to_date,gender,scholarship_holder,age_at_enrollment,international,curricular_units_1st_sem_(credited),curricular_units_1st_sem_(enrolled),curricular_units_1st_sem_(evaluations),curricular_units_1st_sem_(approved),curricular_units_1st_sem_(grade),curricular_units_1st_sem_(without_evaluations),curricular_units_2nd_sem_(credited),curricular_units_2nd_sem_(enrolled),curricular_units_2nd_sem_(evaluations),curricular_units_2nd_sem_(approved),curricular_units_2nd_sem_(grade),curricular_units_2nd_sem_(without_evaluations),unemployment_rate,inflation_rate,gdp,target
0,1,44,1,9085,1,39,150.0,1,37,37,9,8,150.0,1,0,0,1,0,1,21,0,1,6,8,6,12.666667,0,1,6,21,5,12.777778,0,16.2,0.3,-0.92,Graduate
1,1,17,2,9254,1,1,127.0,1,1,1,4,8,122.2,1,0,0,1,1,0,20,0,0,6,10,3,11.666667,0,0,6,12,3,11.000000,0,15.5,2.8,-4.06,Dropout
2,1,1,1,9773,1,1,140.0,1,37,38,9,9,132.1,1,0,0,1,0,1,19,0,0,6,9,4,11.500000,0,0,6,6,5,12.200000,0,7.6,2.6,0.32,Graduate
3,1,1,1,171,1,1,110.0,1,19,2,5,2,117.7,0,0,0,1,1,0,18,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,12.4,0.5,1.79,Enrolled
4,1,39,1,9238,1,19,133.1,1,34,38,0,4,100.0,0,0,0,1,1,0,27,0,0,6,12,0,0.000000,0,0,6,13,1,11.000000,0,9.4,-0.8,-3.12,Dropout


In [24]:
train_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61214 entries, 0 to 61213
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   marital_status                                  61214 non-null  int64  
 1   application_mode                                61214 non-null  int64  
 2   application_order                               61214 non-null  int64  
 3   course                                          61214 non-null  int64  
 4   daytime/evening_attendance                      61214 non-null  int64  
 5   previous_qualification                          61214 non-null  int64  
 6   previous_qualification_(grade)                  61214 non-null  float64
 7   nacionality                                     61214 non-null  int64  
 8   mother's_qualification                          61214 non-null  int64  
 9   father's_qualification                 

In [25]:
X_train = train_clean.drop(columns= "target")
y_train = train_clean.target.copy()

## 4. Transformaton Operations

In [26]:
X_train.columns

Index(['marital_status', 'application_mode', 'application_order', 'course',
       'daytime/evening_attendance', 'previous_qualification',
       'previous_qualification_(grade)', 'nacionality',
       'mother's_qualification', 'father's_qualification',
       'mother's_occupation', 'father's_occupation', 'admission_grade',
       'displaced', 'educational_special_needs', 'debtor',
       'tuition_fees_up_to_date', 'gender', 'scholarship_holder',
       'age_at_enrollment', 'international',
       'curricular_units_1st_sem_(credited)',
       'curricular_units_1st_sem_(enrolled)',
       'curricular_units_1st_sem_(evaluations)',
       'curricular_units_1st_sem_(approved)',
       'curricular_units_1st_sem_(grade)',
       'curricular_units_1st_sem_(without_evaluations)',
       'curricular_units_2nd_sem_(credited)',
       'curricular_units_2nd_sem_(enrolled)',
       'curricular_units_2nd_sem_(evaluations)',
       'curricular_units_2nd_sem_(approved)',
       'curricular_units_2nd_s

### Categorical and Non Categorical variables

In [40]:
def identify_and_convert(df, threshold =10):
    for col in df.columns:
        if df[col].nunique()<= threshold:
            df[col] = df[col].astype('category')
    return df

### 4.1 marital_status

In [37]:
marital_status_transformer = Pipeline(steps=[
    ("grouper", RareLabelEncoder(0.01, replace_with="other", n_categories= 2))
])

marital_status_transformer.fit_transform(X_train.loc[:,["marital_status"]])

TypeError: No categorical variables found in this dataframe. Please check variable format with pandas dtypes.